In [1]:
pip install xarray netcdf4 dask

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.5 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 60.9 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 134.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 79.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 59.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 156.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 KB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 KB 123.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 KB 96.3 MB/s eta 0:00:00


In [2]:
!pip install scipy>=1.12.0 --upgrade -v

In [3]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 2.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import scipy as sp
print(f"NumPy version: {np.__version__}")
print(f"SciPy version: {sp.__version__}")

NumPy version: 2.2.5
SciPy version: 1.15.3


In [5]:
!python -m pip install "dask[distributed]"

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 KB 82.0 MB/s eta 0:00:00


In [6]:
!pip install psutil

Defaulting to user installation because normal site-packages is not writeable


In [7]:
"""
Create Stage-1 ConvLSTM samples for a 3-month lookback → next-month fire classification,
treating static features as extra channels only at the final time step (t=2).

Data Paths (based on your previous scripts and storage):
  • Fire features:   "fire/{year}/{FeatureName}.npz"     # shape [12, num_patches, 128, 128]
  • Climate features:"climate/{year}/{FeatureName}.npz"  # shape [12, num_patches, 128, 128]
  • Static features: "static_patches/{FeatureName}.npz"  # shape [num_patches, 128, 128]
  • Labels (binary): "Patch_Labels/{year}/BurnedArea_Labels.npz"  # shape [12, num_patches]

Script Outline:
  1) We loop over valid (year, month) pairs where we have (t-2, t-1, t) input data and (t+1) label.
  2) For each dynamic directory ("fire", "climate"), load all .npz features for each needed month.
  3) Combine these monthly patches into [3, dynamic_channels, num_patches, 128, 128].
  4) Load static features once. For time steps 0 & 1, no static channels. For time=2, append static along channels.
  5) Load label at (t+1) → shape [num_patches].
  6) Save final sample as .npz: shape [num_patches, channels, time=3, 128, 128], plus label array [num_patches].
  7) Parallelize over months to handle large data efficiently (450GB RAM).

Adjust or extend to match your exact file naming & feature sets.
"""

import os
import numpy as np
import multiprocessing as mp
from tqdm.auto import tqdm
import time
import gc

# Customize these as needed
START_YEAR = 2005
END_YEAR   = 2022
DYNAMIC_DIRS = ["fire", "climate"]     # Where monthly dynamic features live
STATIC_DIR   = "static_patches"        # Where static .npz live
LABELS_ROOT  = "Patch_Labels"          # Where monthly labels live
OUTPUT_DIR   = "Stage1_Samples"        # Where final samples go

os.makedirs(OUTPUT_DIR, exist_ok=True)

def valid_year_months():
    """
    Generate (year, month) combos for which we have (t-2, t-1, t) input and (t+1) label.
    - Earliest possible t is START_YEAR, March  (since t-2 => January).
    - Latest possible t is END_YEAR, November (since t+1 => December).
    """
    pairs = []
    total_months = (END_YEAR - START_YEAR + 1) * 12
    for y in range(START_YEAR, END_YEAR + 1):
        for m in range(1, 13):
            g = (y - START_YEAR)*12 + (m - 1)  # zero-based
            if g >= 2 and g <= total_months - 2:
                pairs.append((y, m))
    return pairs

def get_global_index(year, month):
    """Convert (year, month) to zero-based index from (START_YEAR,1)=0 up to (END_YEAR,12)."""
    return (year - START_YEAR)*12 + (month - 1)

def get_year_month(global_idx):
    """Inverse of get_global_index -> (year, month)."""
    y = global_idx // 12 + START_YEAR
    m = global_idx % 12 + 1
    return (y, m)

def load_single_month_feature(dir_path, year, feature_name, month_idx):
    """
    Loads one month's data from e.g. "dir_path/{year}/{feature_name}.npz".
    Shape inside: [12, num_patches, 128, 128].
    month_idx is 0-based (Jan=0..Dec=11).
    Returns float32 array [num_patches, 128, 128].
    """
    fpath = os.path.join(dir_path, str(year), f"{feature_name}.npz")
    if not os.path.exists(fpath):
        raise FileNotFoundError(f"File not found: {fpath}")
    with np.load(fpath, allow_pickle=True) as npz:
        data_4d = npz["data"]  # shape [12, num_patches, 128, 128]
    if month_idx < 0 or month_idx >= data_4d.shape[0]:
        raise ValueError(f"Invalid month_idx={month_idx} for file={fpath}")
    return data_4d[month_idx]  # [num_patches, 128, 128]

def load_labels(year, month):
    """
    Loads binary labels from Patch_Labels/{year}/BurnedArea_Labels.npz,
    shape [12, num_patches]. month is 1-based, so index=month-1.
    Returns shape [num_patches].
    """
    label_file = os.path.join(LABELS_ROOT, str(year), "BurnedArea_Labels.npz")
    if not os.path.exists(label_file):
        raise FileNotFoundError(f"Label file not found: {label_file}")
    with np.load(label_file, allow_pickle=True) as npz:
        labels_2d = npz["labels"]  # shape [12, num_patches]
    return labels_2d[month - 1]

def load_static_data():
    """
    Loads all static .npz in STATIC_DIR and stacks them along a channel dim:
      final shape: [channels, num_patches, 128, 128].
    Returns an empty array if none found.
    """
    files = sorted([f for f in os.listdir(STATIC_DIR) if f.endswith(".npz")])
    if not files:
        return np.empty((0,0,0,0), dtype=np.float32)

    arrays = []
    num_patches = None
    for f in files:
        fpath = os.path.join(STATIC_DIR, f)
        with np.load(fpath, allow_pickle=True) as npz:
            static_3d = npz["data"]  # [num_patches, 128, 128]
        if num_patches is None:
            num_patches = static_3d.shape[0]
        elif static_3d.shape[0] != num_patches:
            raise ValueError("Mismatch in #patches among static features.")
        arrays.append(static_3d[None, ...])  # => [1, num_patches, 128, 128]

    # Combine into shape [channels, num_patches, 128, 128]
    return np.concatenate(arrays, axis=0).astype(np.float32)

def build_sample(year, month, static_arr):
    """
    Build a single sample chunk for time t=(year,month),
    with 3-month lookback => label at t+1,
    and static channels only at final time index (t=2).
    Saves to Stage1_Samples/samples_YYYY_MM.npz as:
      x_data: [num_patches, (dyn_ch+static_ch), 3, 128, 128]
      y_label: [num_patches]
    """
    # Convert (year, month) -> global index
    t_idx = get_global_index(year, month)
    # We'll load data for t-2, t-1, t => in_months, label from t+1 => label_month
    in_indices = [t_idx-2, t_idx-1, t_idx]
    label_idx = t_idx + 1
    in_ym = [get_year_month(i) for i in in_indices]
    label_ym = get_year_month(label_idx)

    # 1) Gather dynamic features from "fire" + "climate"
    # Let's discover feature files in each directory's {year} folder
    # Then load each feature for each of the 3 months -> shape [3, num_patches, 128, 128], stack them in channel dimension
    dynamic_stacks = []  # will hold arrays [3, num_patches, 128, 128] for each feature
    dyn_file_list = []
    for ddir in DYNAMIC_DIRS:
        y_dir = os.path.join(ddir, str(in_ym[0][0]))  # e.g. "fire/2005"
        if not os.path.isdir(ddir):
            raise FileNotFoundError(f"Dynamic directory not found: {ddir}")
        # Collect .npz in that directory's year subfolder
        if os.path.isdir(y_dir):
            for f in os.listdir(y_dir):
                if f.endswith(".npz"):
                    # e.g. "E0.npz", "NDVI.npz" etc.
                    dyn_file_list.append((ddir, f.replace(".npz","")))

    # Remove duplicates if any, and sort for consistency
    dyn_file_list = list(set(dyn_file_list))
    dyn_file_list.sort()

    # For each feature, load t-2, t-1, t
    dynamic_data_list = []
    for (dir_path, feat_name) in dyn_file_list:
        # Each feature -> [3, num_patches, 128, 128]
        three_stack = []
        for (yy, mm) in in_ym:
            # 0-based index for the month data
            m_idx = mm - 1
            arr = load_single_month_feature(dir_path, yy, feat_name, m_idx)
            three_stack.append(arr[None, ...])  # shape [1, num_patches, 128, 128]
        feat_3d = np.concatenate(three_stack, axis=0)  # => [3, num_patches, 128, 128]
        dynamic_data_list.append(feat_3d)

    if dynamic_data_list:
        # Concatenate along a new "channel" dimension => final shape [3, dyn_channels, num_patches, 128, 128]
        # But each item is [3, num_patches, 128, 128], so we stack them along axis=1
        # We'll do the standard trick:
        # 1) combine them along axis=0 => shape [ (n_features*3), num_patches, 128, 128]
        # 2) reshape => [n_features, 3, num_patches, 128, 128]
        merged = np.concatenate(dynamic_data_list, axis=0)  # shape [n_feats*3, num_patches, 128, 128]
        n_feats = len(dynamic_data_list)
        merged = merged.reshape(n_feats, 3, -1, 128, 128)   # => [n_feats, 3, num_patches, 128, 128]
        merged = np.swapaxes(merged, 0, 1)                 # => [3, n_feats, num_patches, 128, 128]
    else:
        # No dynamic data found
        merged = np.zeros((3, 0, 0, 128, 128), dtype=np.float32)

    # merged => [3, dyn_channels, num_patches, 128, 128]
    dyn_channels = merged.shape[1]
    num_patches  = merged.shape[2]

    # 2) Insert static at final time step => t=2
    # static_arr: [static_channels, num_patches, 128, 128]
    static_ch = static_arr.shape[0] if static_arr.size > 0 else 0

    # final_data => [3, dyn_channels+static_ch, num_patches, 128, 128]
    final_data = np.zeros(
        (3, dyn_channels + static_ch, num_patches, 128, 128),
        dtype=np.float32
    )

    # Place dynamic data first
    final_data[:, :dyn_channels, :, :, :] = merged

    # Place static in final time step = 2, channels [dyn_channels : dyn_channels+static_ch]
    if static_ch > 0:
        # static_arr => [static_ch, num_patches, 128, 128]
        # final_data[2, dyn_channels: ...] => shape [static_ch, num_patches, 128, 128]
        final_data[2, dyn_channels:dyn_channels+static_ch, :, :, :] = static_arr

    # 3) Load label at t+1 => label_ym
    labels_1d = load_labels(label_ym[0], label_ym[1])  # shape [num_patches]

    # 4) Reorder to [num_patches, channels, time=3, 128, 128]
    # Currently => [time=3, ch, patches, 128, 128]
    final_data = np.swapaxes(final_data, 1, 2)   # => [3, num_patches, (dyn+st), 128, 128]
    final_data = np.swapaxes(final_data, 0, 1)   # => [num_patches, 3, (dyn+st), 128, 128]
    final_data = np.swapaxes(final_data, 1, 2)   # => [num_patches, (dyn+st), 3, 128, 128]

    out_name = f"samples_{year:04d}_{month:02d}.npz"
    out_path = os.path.join(OUTPUT_DIR, out_name)

    # Save
    metadata = {
        "year": year,
        "month": month,
        "num_patches": num_patches,
        "time_steps": 3,
        "dyn_channels": dyn_channels,
        "static_channels": static_ch,
        "label_year_month": label_ym,
        "description": "3-month input => next-month label; static at final time index only."
    }

    np.savez_compressed(
        out_path,
        x_data=final_data.astype(np.float32),
        y_label=labels_1d.astype(np.uint8),
        metadata=metadata
    )

    del merged, final_data, labels_1d
    gc.collect()

def process_month(arg):
    """Wrapper for multiprocessing: arg=(year, month, static_arr)."""
    (year, month, static_arr) = arg
    try:
        build_sample(year, month, static_arr)
        return (year, month, "OK")
    except Exception as e:
        return (year, month, f"ERROR: {str(e)}")

def main():
    start_time = time.time()

    # 1) Load static features once
    print("Loading static data into memory...")
    static_arr = load_static_data()
    print(f"Static shape = {static_arr.shape} [channels, patches, 128, 128]")

    # 2) Create list of valid (year, month)
    vm_list = valid_year_months()
    print(f"{len(vm_list)} valid (year, month) combos for 3-month input => next-month label.")

    # Prepare args for parallel map
    # We'll bundle static_arr in the argument so each process can access it read-only.
    tasks = [(y, m, static_arr) for (y, m) in vm_list]

    # 3) Parallel processing
    num_workers = min(mp.cpu_count(), 8)
    print(f"Starting pool with {num_workers} workers...")
    with mp.Pool(num_workers) as pool:
        results = list(
            tqdm(pool.imap_unordered(process_month, tasks), total=len(tasks), desc="Processing Months")
        )

    # 4) Summarize
    ok_count = sum(1 for r in results if r[2] == "OK")
    err_count = len(results) - ok_count
    print(f"\nCompleted: {ok_count} month-chunks OK, {err_count} errors.")
    if err_count > 0:
        print("Errors:")
        for r in results:
            if r[2].startswith("ERROR"):
                print(r)

    elapsed = time.time() - start_time
    print(f"\nAll done. Generated Stage-1 samples in '{OUTPUT_DIR}'.")
    print(f"Total time: {elapsed:.2f} s ({elapsed/60:.2f} min).")

if __name__ == "__main__":
    mp.freeze_support()
    main()


/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading static data into memory...
Static shape = (11, 2051, 128, 128) [channels, patches, 128, 128]
213 valid (year, month) combos for 3-month input => next-month label.
Starting pool with 8 workers...


Processing Months: 100%|██████████| 213/213 [1:56:24<00:00, 32.79s/it]  


Completed: 213 month-chunks OK, 0 errors.

All done. Generated Stage-1 samples in 'Stage1_Samples'.
Total time: 6988.96 s (116.48 min).


Will read chunk files from: Stage1_Samples
Splits saved to: splits
Train years: {2016, 2017, 2018, 2019, 2020, 2012, 2013, 2014, 2015}
Val year:    {2021}
Test years:  {2010, 2011, 2022}


In [7]:

# --- Cell [1.1]: Imports & Config ---
import os
import json
import numpy as np
import random
from tqdm.auto import tqdm

# Fixed random seed for reproducibility (if further random ops are needed)
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

SAMPLES_DIR = "Stage1_Samples"  # where your 213 chunk files are
SPLITS_DIR = "splits"           # output directory for split info
os.makedirs(SPLITS_DIR, exist_ok=True)

TRAIN_YEARS = set(range(2012, 2021))  # 2012..2020 inclusive
VAL_YEARS   = {2021}
TEST_YEARS  = {2010, 2011, 2022}

print(f"Will read chunk files from: {SAMPLES_DIR}")
print(f"Splits saved to: {SPLITS_DIR}")
print(f"Train years: {TRAIN_YEARS}")
print(f"Val year:    {VAL_YEARS}")
print(f"Test years:  {TEST_YEARS}")


# --- Cell [1.2]: Utility Functions ---

def parse_year_month_from_filename(fname):
    """
    Expects a filename like 'samples_YYYY_MM.npz'.
    Returns (year, month) as integers.
    Raises ValueError if the format is unexpected.
    """
    # Remove extension
    base = fname.replace(".npz", "")
    # base should be 'samples_YYYY_MM'
    parts = base.split("_")
    if len(parts) != 3:
        raise ValueError(f"Filename '{fname}' does not match 'samples_YYYY_MM.npz'")
    # parts[0] = 'samples'
    # parts[1] = YYYY
    # parts[2] = MM
    try:
        year = int(parts[1])
        month = int(parts[2])
    except:
        raise ValueError(f"Cannot parse year/month from '{fname}'")
    return year, month


def load_labels_and_count_fire(npz_path):
    """
    Opens 'npz_path', which should contain:
      - y_label: shape [num_patches], values in {0,1}
    Returns a tuple (count_fire, count_non_fire).
    """
    with np.load(npz_path, allow_pickle=True) as data:
        y = data["y_label"]  # shape [num_patches], 0 or 1
    y = y.astype(np.int32)
    count_fire = int(np.sum(y == 1))
    count_non_fire = int(np.sum(y == 0))
    return count_fire, count_non_fire

# --- Cell [1.3]: Identify Splits, Count Class Imbalance ---

def split_and_analyze(samples_dir):
    # Initialize containers
    train_files = []
    val_files   = []
    test_files  = []

    # Accumulators for class imbalance
    # We'll store total fire / non-fire patch counts for each split
    train_fire = train_nonfire = 0
    val_fire   = val_nonfire   = 0
    test_fire  = test_nonfire  = 0
    
    # Gather all chunk files
    chunk_files = [f for f in os.listdir(samples_dir) if f.endswith(".npz")]
    chunk_files.sort()

    print(f"Found {len(chunk_files)} candidate chunk files in '{samples_dir}'.")

    for fname in tqdm(chunk_files, desc="Splitting & Counting"):
        # Parse (year, month)
        year, month = parse_year_month_from_filename(fname)

        # Decide which split it belongs to
        if year in TRAIN_YEARS:
            split_name = "train"
        elif year in VAL_YEARS:
            split_name = "val"
        elif year in TEST_YEARS:
            split_name = "test"
        else:
            # According to your rule, we skip chunks outside those sets (e.g. 2005..2009)
            # Or place them in test if you want. We'll skip them:
            continue
        
        # Load label distribution
        npz_path = os.path.join(samples_dir, fname)
        c_fire, c_nonfire = load_labels_and_count_fire(npz_path)

        # Accumulate counts
        if split_name == "train":
            train_files.append(fname)
            train_fire    += c_fire
            train_nonfire += c_nonfire
        elif split_name == "val":
            val_files.append(fname)
            val_fire    += c_fire
            val_nonfire += c_nonfire
        else:  # test
            test_files.append(fname)
            test_fire    += c_fire
            test_nonfire += c_nonfire

    # Summaries
    train_total = train_fire + train_nonfire
    val_total   = val_fire + val_nonfire
    test_total  = test_fire + test_nonfire

    # Prepare results dictionary
    results = {
        "splits": {
            "train": train_files,
            "validation": val_files,
            "test": test_files
        },
        "class_distribution": {
            "train": {
                "fire_count": train_fire,
                "non_fire_count": train_nonfire,
                "total_patches": train_total,
                "fire_ratio": float(train_fire / train_total) if train_total>0 else 0.0
            },
            "validation": {
                "fire_count": val_fire,
                "non_fire_count": val_nonfire,
                "total_patches": val_total,
                "fire_ratio": float(val_fire / val_total) if val_total>0 else 0.0
            },
            "test": {
                "fire_count": test_fire,
                "non_fire_count": test_nonfire,
                "total_patches": test_total,
                "fire_ratio": float(test_fire / test_total) if test_total>0 else 0.0
            }
        }
    }

    return results

# Run the splitting and analysis
split_info = split_and_analyze(SAMPLES_DIR)


# --- Cell [1.4]: Save Outputs & Final Summary ---

import json

# 1) Save the entire split info in a single JSON
split_info_path = os.path.join(SPLITS_DIR, "split_info.json")
with open(split_info_path, "w") as f:
    json.dump(split_info, f, indent=2)

# 2) Optionally, we can save the class distribution alone in a separate JSON
dist_info_path = os.path.join(SPLITS_DIR, "class_distribution.json")
with open(dist_info_path, "w") as f:
    json.dump(split_info["class_distribution"], f, indent=2)

print(f"Saved split info to:\n  {split_info_path}\n  {dist_info_path}")

# 3) Print a quick summary
cdist = split_info["class_distribution"]
print("\n=== Class Distribution Summary ===")
print(f"Train => Fire: {cdist['train']['fire_count']}, Non-Fire: {cdist['train']['non_fire_count']}, "
      f"Ratio: {cdist['train']['fire_ratio']:.3f}, Total: {cdist['train']['total_patches']}")
print(f"Val   => Fire: {cdist['validation']['fire_count']}, Non-Fire: {cdist['validation']['non_fire_count']}, "
      f"Ratio: {cdist['validation']['fire_ratio']:.3f}, Total: {cdist['validation']['total_patches']}")
print(f"Test  => Fire: {cdist['test']['fire_count']}, Non-Fire: {cdist['test']['non_fire_count']}, "
      f"Ratio: {cdist['test']['fire_ratio']:.3f}, Total: {cdist['test']['total_patches']}")
print("\nDone.")



Will read chunk files from: Stage1_Samples
Splits saved to: splits
Train years: {2016, 2017, 2018, 2019, 2020, 2012, 2013, 2014, 2015}
Val year:    {2021}
Test years:  {2010, 2011, 2022}
Found 213 candidate chunk files in 'Stage1_Samples'.


Splitting & Counting: 100%|██████████| 213/213 [00:01<00:00, 107.25it/s]

Saved split info to:
  splits/split_info.json
  splits/class_distribution.json

=== Class Distribution Summary ===
Train => Fire: 37298, Non-Fire: 184210, Ratio: 0.168, Total: 221508
Val   => Fire: 3402, Non-Fire: 21210, Ratio: 0.138, Total: 24612
Test  => Fire: 11698, Non-Fire: 60087, Ratio: 0.163, Total: 71785

Done.


In [11]:
!pip install numpy==1.24.3

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 28.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5


In [12]:
!pip install tensorflow==2.14.0

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.8/262.8 MB 29.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 151.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 KB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 154.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 143.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 221.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 202.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.1/216.1 KB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 276.5 MB/s eta 0:00:00


In [8]:
# =================================================================================
# CELL [1]: Imports & Configuration
# =================================================================================
import os
import json
import numpy as np
import random
import tensorflow as tf
from tqdm.auto import tqdm

# Fixed seed for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

SPLITS_DIR   = "splits"          # Where split_info.json is stored
SAMPLES_DIR  = "Stage1_Samples"  # Where the .npz chunk files are
SPLIT_FILE   = "split_info.json"

BATCH_SIZE   = 64
OVERSAMPLE   = True              # Whether to oversample minority class in train
MAX_RAM_USAGE= 1_000_000_000     # ~1GB, adjust to your memory constraints

# Optionally specify data types for tf.data
TF_DTYPE = tf.float32
TF_LABEL_DTYPE = tf.int32

# =================================================================================
# CELL [2]: Load Splits & Check
# =================================================================================

def load_splits(splits_dir=SPLITS_DIR, split_file=SPLIT_FILE):
    """Load split_info.json containing train/val/test file lists."""
    path = os.path.join(splits_dir, split_file)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Split file not found: {path}")
    with open(path, "r") as f:
        info = json.load(f)
    return info

split_info = load_splits()
train_files = split_info["splits"]["train"]      # e.g. list of chunk filenames
val_files   = split_info["splits"]["validation"]
test_files  = split_info["splits"]["test"]

print("Loaded splits from:", os.path.join(SPLITS_DIR, SPLIT_FILE))
print(f"Train files: {len(train_files)}")
print(f"Val   files: {len(val_files)}")
print(f"Test  files: {len(test_files)}")

# =================================================================================
# CELL [3]: Data Generator (Python) With Oversampling
# =================================================================================

def chunk_file_generator(file_list, samples_dir, batch_size, oversample=False, shuffle_files=True):
    """
    A Python generator that:
      1) Shuffles 'file_list' if requested.
      2) Iterates over each chunk file in random order (if shuffle_files=True).
      3) Loads x_data, y_label from the .npz file.
      4) Optionally oversamples minority class.
      5) Yields mini-batches of shape (batch_size, ...).

    Yields:
      (x_batch, y_batch) as NumPy arrays.

    Use:
      gen = chunk_file_generator(train_files, SAMPLES_DIR, BATCH_SIZE, oversample=True)
      for x, y in gen:
          ...

    We'll later wrap this in tf.data.Dataset.from_generator(...)
    """
    # Make a copy so we don’t modify the original list
    files = list(file_list)
    if shuffle_files:
        random.shuffle(files)

    for fname in files:
        # Load chunk data from .npz
        fpath = os.path.join(samples_dir, fname)
        if not os.path.exists(fpath):
            raise FileNotFoundError(f"Missing chunk file: {fpath}")

        with np.load(fpath, allow_pickle=True) as data:
            x_data = data["x_data"]  # shape [N, channels, time=3, 128, 128]
            y_data = data["y_label"] # shape [N]
        
        # Approx memory check
        approx_mem = x_data.size * x_data.itemsize + y_data.size * y_data.itemsize
        if approx_mem > MAX_RAM_USAGE:
            print(f"Warning: chunk {fname} size {approx_mem/1e9:.2f} GB > limit of {MAX_RAM_USAGE/1e9:.2f} GB")

        # Oversample minority class if requested
        if oversample:
            pos_idx = np.where(y_data == 1)[0]
            neg_idx = np.where(y_data == 0)[0]
            num_pos = len(pos_idx)
            num_neg = len(neg_idx)
            if num_pos > 0 and num_neg > 0:
                if num_pos < num_neg:
                    # replicate positives
                    factor = num_neg // num_pos
                    remainder = num_neg % num_pos
                    replicated_idx = np.concatenate(
                        [pos_idx]*factor + 
                        [np.random.choice(pos_idx, remainder, replace=False)] if remainder>0 else []
                    )
                    new_idx = np.concatenate([neg_idx, replicated_idx])
                else:
                    # replicate negatives
                    factor = num_pos // num_neg
                    remainder = num_pos % num_neg
                    replicated_idx = np.concatenate(
                        [neg_idx]*factor + 
                        [np.random.choice(neg_idx, remainder, replace=False)] if remainder>0 else []
                    )
                    new_idx = np.concatenate([pos_idx, replicated_idx])
                
                # shuffle new index set
                np.random.shuffle(new_idx)
                x_data = x_data[new_idx]
                y_data = y_data[new_idx]

        # Build local index array for mini-batch sampling
        indices = np.arange(x_data.shape[0])
        np.random.shuffle(indices)  # shuffle to avoid chunk-level order bias

        start = 0
        while start < len(indices):
            end = min(start + batch_size, len(indices))
            batch_idx = indices[start:end]
            start = end

            x_batch = x_data[batch_idx]
            y_batch = y_data[batch_idx]
            yield x_batch, y_batch

        # Clean up this chunk from RAM
        del x_data, y_data

# =================================================================================
# CELL [4]: Build tf.data Datasets
# =================================================================================

def make_dataset(file_list, samples_dir, batch_size, oversample, shuffle_files, name="dataset"):
    """
    Wraps chunk_file_generator into a tf.data.Dataset.
    We rely on from_generator(...), specifying output_types and output_shapes
    for TensorFlow.
    """
    # 1) Define a generator function with no args, capturing the local parameters
    def gen():
        return chunk_file_generator(
            file_list    = file_list,
            samples_dir  = samples_dir,
            batch_size   = batch_size,
            oversample   = oversample,
            shuffle_files= shuffle_files
        )

    # 2) Peek into one sample to define shapes
    #    We'll do a small hack: run the generator once
    #    or we can just define None for flexible batch dimension.
    #    We'll do shape (None, channels, time=3, 128, 128), label => (None,).
    #    So we won't do strict shape checks at the dataset level, we allow variable batch size for the last batch.
    output_types = (TF_DTYPE, TF_LABEL_DTYPE)
    # We'll guess the shapes at runtime, but we do know "channels"=unknown, "time"=3, "height"=128, "width"=128
    # We'll set them to (None, None, None, None, None) to avoid strict shape constraints.
    output_shapes = (
        tf.TensorShape([None, None, None, None, None]),  # x_batch
        tf.TensorShape([None])                           # y_batch
    )

    dataset = tf.data.Dataset.from_generator(
        gen,
        output_types=output_types,
        output_shapes=output_shapes
    )

    # Return a repeatable dataset if needed (the user can call dataset.repeat() or pass steps_per_epoch).
    # We'll just return it as is for now.
    return dataset

# Create train/val/test datasets using the above function
train_ds = make_dataset(
    file_list     = train_files,
    samples_dir   = SAMPLES_DIR,
    batch_size    = BATCH_SIZE,
    oversample    = OVERSAMPLE,
    shuffle_files = True,
    name="train"
)

val_ds = make_dataset(
    file_list     = val_files,
    samples_dir   = SAMPLES_DIR,
    batch_size    = BATCH_SIZE,
    oversample    = False,  # typically no oversampling in validation
    shuffle_files = False,
    name="val"
)

test_ds = make_dataset(
    file_list     = test_files,
    samples_dir   = SAMPLES_DIR,
    batch_size    = BATCH_SIZE,
    oversample    = False,  # no oversampling in test
    shuffle_files = False,
    name="test"
)

print("tf.data Datasets created for train, val, and test.")


Loaded splits from: splits/split_info.json
Train files: 108
Val   files: 12
Test  files: 35
tf.data Datasets created for train, val, and test.


In [3]:
pip install tensorflow-addons==0.20.0

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tensorflow-addons==0.20.0 (from versions: none)
ERROR: No matching distribution found for tensorflow-addons==0.20.0
Note: you may need to restart the kernel to use updated packages.


In [5]:
# ===========================================================================================
# CELL [5]: Enhanced ConvLSTM Training Demo (No TensorFlow Addons)
#           - Custom Focal Loss Implementation
#           - Gradient clipping
#           - Debug printing of random slices for NaN analysis & label check
#           - Stronger ConvLSTM architecture
# ===========================================================================================

import os
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# Increase memory threshold if your chunk files are ~9.68 GB
MAX_RAM_USAGE = 15_000_000_000  # ~15 GB; adjust if you have 450GB

# 1) Custom Focal Loss Implementation
def sigmoid_focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):
    """
    Custom focal loss for binary classification.
      - y_true, y_pred are same shape [batch].
      - 'gamma' focuses on hard, misclassified examples.
      - 'alpha' is weighting for positive class.
    """
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)  # avoid log(0)
    y_true = tf.cast(y_true, y_pred.dtype)

    # cross-entropy
    ce = tf.keras.backend.binary_crossentropy(y_true, y_pred, from_logits=False)
    # p_t: prob assigned to true class
    p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
    # alpha weighting
    alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
    # modulating factor (1 - p_t)^gamma
    modulating_factor = tf.pow((1.0 - p_t), gamma)

    focal_loss = alpha_factor * modulating_factor * ce
    return tf.reduce_mean(focal_loss)

# ----------------------------------------------------------------------------
# 2) Updated chunk_file_generator with debug prints & oversampling
# ----------------------------------------------------------------------------

def debug_print_sample_slices(x_data, y_data, num_checks=3):
    """
    Print random patch slices for NaN ratio & label.
    x_data shape: [N, channels, time=3, 128, 128]
    y_data shape: [N]
    """
    if x_data.shape[0] == 0:
        return
    # We'll sample a few patch indices at random
    indices = np.random.choice(x_data.shape[0], size=min(num_checks, x_data.shape[0]), replace=False)
    for i in indices:
        c = np.random.randint(0, x_data.shape[1])  # channels
        t = np.random.randint(0, x_data.shape[2])  # time steps
        patch_slice = x_data[i, c, t]  # shape: [128, 128]
        nan_count = np.isnan(patch_slice).sum()
        total_cells = 128 * 128
        nan_pct = (nan_count / total_cells) * 100.0
        label_val = y_data[i]
        print(f"  Debug patch idx={i}, channel={c}, time={t}, NaN%={nan_pct:.2f}, label={label_val}")

OVERSAMPLE = True

def chunk_file_generator(file_list, samples_dir, batch_size, oversample=False, shuffle_files=True, debug_checks=2):
    files = list(file_list)
    if shuffle_files:
        random.shuffle(files)

    for fname in files:
        fpath = os.path.join(samples_dir, fname)
        if not os.path.exists(fpath):
            raise FileNotFoundError(f"Missing chunk file: {fpath}")

        with np.load(fpath, allow_pickle=True) as data:
            x_data = data["x_data"]
            y_data = data["y_label"]

        approx_mem = x_data.size * x_data.itemsize + y_data.size * y_data.itemsize
        if approx_mem > MAX_RAM_USAGE:
            print(f"Warning: chunk {fname} size {approx_mem/1e9:.2f} GB > limit of {MAX_RAM_USAGE/1e9:.2f} GB")

        # Print random debug slices
        print(f"\n>>> Debug Info: file={fname}, shape={x_data.shape}")
        debug_print_sample_slices(x_data, y_data, num_checks=debug_checks)

        # Oversample minority class if requested
        if oversample:
            pos_idx = np.where(y_data == 1)[0]
            neg_idx = np.where(y_data == 0)[0]
            num_pos = len(pos_idx)
            num_neg = len(neg_idx)
            if num_pos > 0 and num_neg > 0:
                if num_pos < num_neg:
                    factor = num_neg // num_pos
                    remainder = num_neg % num_pos
                    replicated_idx = np.concatenate(
                        [pos_idx]*factor +
                        ([np.random.choice(pos_idx, remainder, replace=False)] if remainder>0 else [])
                    )
                    new_idx = np.concatenate([neg_idx, replicated_idx])
                else:
                    factor = num_pos // num_neg
                    remainder = num_pos % num_neg
                    replicated_idx = np.concatenate(
                        [neg_idx]*factor +
                        ([np.random.choice(neg_idx, remainder, replace=False)] if remainder>0 else [])
                    )
                    new_idx = np.concatenate([pos_idx, replicated_idx])
                np.random.shuffle(new_idx)
                x_data = x_data[new_idx]
                y_data = y_data[new_idx]

        # Shuffle local indices
        indices = np.arange(x_data.shape[0])
        np.random.shuffle(indices)

        start = 0
        while start < len(indices):
            end = min(start + batch_size, len(indices))
            batch_idx = indices[start:end]
            start = end

            yield x_data[batch_idx], y_data[batch_idx]

        del x_data, y_data

def make_dataset(file_list, samples_dir, batch_size, oversample=True, shuffle_files=True):
    def gen():
        return chunk_file_generator(
            file_list=file_list,
            samples_dir=samples_dir,
            batch_size=batch_size,
            oversample=oversample,
            shuffle_files=shuffle_files,
            debug_checks=3
        )
    output_types = (tf.float32, tf.int32)
    output_shapes = (
        tf.TensorShape([None, None, 3, 128, 128]),  # x_batch
        tf.TensorShape([None])                      # y_batch
    )
    ds = tf.data.Dataset.from_generator(gen, output_types=output_types, output_shapes=output_shapes)
    return ds

def build_stronger_convlstm(channels):
    """
    Deeper ConvLSTM model with larger filters and BN for better capacity.
    """
    inp = layers.Input(shape=(channels, 3, 128, 128), name="Input_ConvLSTM")
    
    x = layers.Permute((2, 3, 4, 1), name="Permute")(inp)
    # => (time=3, 128, 128, channels)

    x = layers.ConvLSTM2D(
        filters=32, kernel_size=(3,3), padding="same", activation="tanh",
        return_sequences=True, name="ConvLSTM_1"
    )(x)
    x = layers.BatchNormalization(name="BN_1")(x)

    x = layers.ConvLSTM2D(
        filters=32, kernel_size=(3,3), padding="same", activation="tanh",
        return_sequences=False, name="ConvLSTM_2"
    )(x)
    x = layers.BatchNormalization(name="BN_2")(x)

    x = layers.Flatten(name="Flatten")(x)
    x = layers.Dense(128, activation="relu", name="Dense_128")(x)
    x = layers.Dropout(0.3, name="Dropout")(x)
    out = layers.Dense(1, activation="sigmoid", name="Output")(x)

    model = models.Model(inp, out, name="Stronger_ConvLSTM")
    return model

# Detect channels from first train file
if len(train_files) == 0:
    raise ValueError("No training files in 'train' split.")
first_train_file = os.path.join(SAMPLES_DIR, train_files[0])
with np.load(first_train_file, allow_pickle=True) as data:
    x_sample = data["x_data"]
detected_channels = x_sample.shape[1]
print(f"Detected channel dimension = {detected_channels}")

model = build_stronger_convlstm(detected_channels)

# Use the custom focal loss
def custom_focal_loss(y_true, y_pred):
    return sigmoid_focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=5.0),
    loss=custom_focal_loss,
    metrics=["accuracy"]
)
model.summary()

EPOCHS = 2
STEPS_PER_EPOCH = 50
VAL_STEPS = 10
TEST_STEPS = 10
train_ds = make_dataset(train_files, SAMPLES_DIR, batch_size=64, oversample=True, shuffle_files=True)
val_ds   = make_dataset(val_files,   SAMPLES_DIR, batch_size=64, oversample=False, shuffle_files=False)
test_ds  = make_dataset(test_files,  SAMPLES_DIR, batch_size=64, oversample=False, shuffle_files=False)

print("\n>>> Starting demonstration training with custom focal loss + oversampling...")
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=val_ds,
    validation_steps=VAL_STEPS
)

print("\n>>> Evaluating on test dataset...")
test_result = model.evaluate(test_ds, steps=TEST_STEPS, return_dict=True)
print("Test metrics:", test_result)

print("\nAll done. Now using a custom focal loss (no tensorflow_addons)!")


NameError: name 'train_files' is not defined

In [2]:
# ===========================================================================================
# FINAL COMPLETE SCRIPT (All in One Cell)
# ===========================================================================================
import os
import json
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# ----------------------------------------------------------------------------
# [1] Configuration & Globals
# ----------------------------------------------------------------------------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

SPLITS_DIR   = "splits"          # Folder containing split_info.json
SPLIT_FILE   = "split_info.json" # The split file name
SAMPLES_DIR  = "Stage1_Samples"  # Where chunk .npz files (samples_YYYY_MM.npz) live

MAX_RAM_USAGE = 15_000_000_000  # ~15 GB memory threshold
BATCH_SIZE    = 64

EPOCHS          = 2
STEPS_PER_EPOCH = 50
VAL_STEPS       = 10
TEST_STEPS      = 10

# ----------------------------------------------------------------------------
# [2] Load Splits
# ----------------------------------------------------------------------------
def load_splits(splits_dir=SPLITS_DIR, split_file=SPLIT_FILE):
    """Load split_info.json containing train/val/test chunk file lists."""
    path = os.path.join(splits_dir, split_file)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Split file not found: {path}")
    with open(path, "r") as f:
        info = json.load(f)
    return info

split_info = load_splits()
train_files = split_info["splits"]["train"]
val_files   = split_info["splits"]["validation"]
test_files  = split_info["splits"]["test"]

print("[INFO] Loaded split info from:", os.path.join(SPLITS_DIR, SPLIT_FILE))
print(f"  => Train files: {len(train_files)}")
print(f"  => Val   files: {len(val_files)}")
print(f"  => Test  files: {len(test_files)}")

# ----------------------------------------------------------------------------
# [3] Generator with Oversampling, Debug Prints, Memory Check
# ----------------------------------------------------------------------------
def debug_print_sample_slices(x_data, y_data, num_checks=3):
    """Print random patch slices for NaN ratio & label."""
    if x_data.shape[0] == 0:
        return
    idx = np.random.choice(x_data.shape[0], size=min(num_checks, x_data.shape[0]), replace=False)
    for i in idx:
        c = np.random.randint(0, x_data.shape[1])  # random channel
        t = np.random.randint(0, x_data.shape[2])  # random time step
        patch_slice = x_data[i, c, t]  # shape: [128,128]
        nan_count = np.isnan(patch_slice).sum()
        total_cells = 128 * 128
        nan_pct = (nan_count / total_cells) * 100.0
        label_val = y_data[i]
        print(f"   Debug patch idx={i}, channel={c}, time={t}, NaN%={nan_pct:.2f}, label={label_val}")

def chunk_file_generator(file_list, samples_dir, batch_size, oversample=True, shuffle_files=True, debug_checks=3):
    """
    Loads each chunk file in random order, prints debug slices, optionally oversamples
    the minority class, and yields mini-batches (x_batch, y_batch).
    """
    files = list(file_list)
    if shuffle_files:
        random.shuffle(files)

    for fname in files:
        fpath = os.path.join(samples_dir, fname)
        if not os.path.exists(fpath):
            raise FileNotFoundError(f"Chunk file not found: {fpath}")

        # Load data from .npz
        with np.load(fpath, allow_pickle=True) as data:
            x_data = data["x_data"]  # shape: [N, channels, time=3, 128,128]
            y_data = data["y_label"] # shape: [N]

        # Approx memory check
        approx_mem = x_data.size * x_data.itemsize + y_data.size * y_data.itemsize
        if approx_mem > MAX_RAM_USAGE:
            print(f"[Warning] chunk {fname} size {approx_mem/1e9:.2f} GB > limit of {MAX_RAM_USAGE/1e9:.2f} GB")

        print(f"\n[DEBUG] Loading {fname}, x_data.shape={x_data.shape}")
        debug_print_sample_slices(x_data, y_data, num_checks=debug_checks)

        # Replace NaNs with 0.0 to avoid nan in forward pass
        x_data = np.nan_to_num(x_data, nan=0.0)

        # Oversample minority class
        if oversample:
            pos_idx = np.where(y_data == 1)[0]
            neg_idx = np.where(y_data == 0)[0]
            num_pos = len(pos_idx)
            num_neg = len(neg_idx)
            if num_pos > 0 and num_neg > 0:
                if num_pos < num_neg:
                    factor = num_neg // num_pos
                    remainder = num_neg % num_pos
                    replicated_idx = np.concatenate(
                        [pos_idx]*factor + 
                        ([np.random.choice(pos_idx, remainder, replace=False)] if remainder>0 else [])
                    )
                    new_idx = np.concatenate([neg_idx, replicated_idx])
                else:
                    factor = num_pos // num_neg
                    remainder = num_pos % num_neg
                    replicated_idx = np.concatenate(
                        [neg_idx]*factor +
                        ([np.random.choice(neg_idx, remainder, replace=False)] if remainder>0 else [])
                    )
                    new_idx = np.concatenate([pos_idx, replicated_idx])
                np.random.shuffle(new_idx)
                x_data = x_data[new_idx]
                y_data = y_data[new_idx]

        # Shuffle local indices for mini-batch iteration
        indices = np.arange(x_data.shape[0])
        np.random.shuffle(indices)

        start = 0
        while start < len(indices):
            end = min(start + batch_size, len(indices))
            batch_idx = indices[start:end]
            start = end

            yield x_data[batch_idx], y_data[batch_idx]

        del x_data, y_data

def make_dataset(file_list, samples_dir, batch_size, oversample=True, shuffle_files=True):
    """Wrap chunk_file_generator in a tf.data.Dataset."""
    def gen():
        return chunk_file_generator(
            file_list=file_list,
            samples_dir=samples_dir,
            batch_size=batch_size,
            oversample=oversample,
            shuffle_files=shuffle_files,
            debug_checks=3
        )
    out_types = (tf.float32, tf.int32)
    out_shapes= (
        tf.TensorShape([None, None, 3, 128, 128]),
        tf.TensorShape([None])
    )
    return tf.data.Dataset.from_generator(gen, output_types=out_types, output_shapes=out_shapes)

# ----------------------------------------------------------------------------
# [4] Stronger ConvLSTM + Custom Focal Loss
# ----------------------------------------------------------------------------
def build_stronger_convlstm(channels):
    """
    Deeper ConvLSTM model with more capacity:
      - 2 stacked ConvLSTM layers w/ 32 filters
      - BN, Flatten, Dense(128), Dropout
      - Sigmoid output
    Expects input shape: (channels, 3, 128, 128).
    """
    inp = layers.Input(shape=(channels, 3, 128, 128), name="Input_ConvLSTM")

    # Permute => (batch, time=3, height=128, width=128, channels)
    x = layers.Permute((2, 3, 4, 1), name="Permute")(inp)

    x = layers.ConvLSTM2D(
        filters=32, kernel_size=(3, 3), padding="same", 
        activation="tanh", return_sequences=True, name="ConvLSTM_1"
    )(x)
    x = layers.BatchNormalization(name="BN_1")(x)

    x = layers.ConvLSTM2D(
        filters=32, kernel_size=(3, 3), padding="same", 
        activation="tanh", return_sequences=False, name="ConvLSTM_2"
    )(x)
    x = layers.BatchNormalization(name="BN_2")(x)

    # Flatten -> Dense -> Dropout -> Output
    x = layers.Flatten(name="Flatten")(x)
    x = layers.Dense(128, activation="relu", name="Dense_128")(x)
    x = layers.Dropout(0.3, name="Dropout")(x)
    out = layers.Dense(1, activation="sigmoid", name="Output")(x)

    return models.Model(inp, out, name="Stronger_ConvLSTM")

def custom_sigmoid_focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):
    """
    Custom focal loss for binary classification:
      - Helps w/ class imbalance
      - (gamma=2.0, alpha=0.25) typical values
    """
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
    y_true = tf.cast(y_true, y_pred.dtype)

    ce = tf.keras.backend.binary_crossentropy(y_true, y_pred, from_logits=False)
    p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
    alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
    modulating_factor = tf.pow(1.0 - p_t, gamma)
    return tf.reduce_mean(alpha_factor * modulating_factor * ce)

# ----------------------------------------------------------------------------
# [5] Build Model, Datasets & Train
# ----------------------------------------------------------------------------

# 1) Detect channel dimension from first train file
if len(train_files) == 0:
    raise ValueError("No training files in 'train' split. Check your split_info.json.")

first_train_file = os.path.join(SAMPLES_DIR, train_files[0])
if not os.path.exists(first_train_file):
    raise FileNotFoundError(f"Cannot find chunk file: {first_train_file}")

with np.load(first_train_file, allow_pickle=True) as data:
    x_sample = data["x_data"]
detected_channels = x_sample.shape[1]
print(f"[INFO] Detected channel dimension = {detected_channels}")

# 2) Build the model
model = build_stronger_convlstm(detected_channels)
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4, clipnorm=5.0),
    loss=lambda yt, yp: custom_sigmoid_focal_loss(yt, yp, gamma=2.0, alpha=0.25),
    metrics=["accuracy"]
)
model.summary()

# 3) Create tf.data Datasets for train, val, test
train_ds = make_dataset(train_files, SAMPLES_DIR, BATCH_SIZE, oversample=True,  shuffle_files=True)
val_ds   = make_dataset(val_files,   SAMPLES_DIR, BATCH_SIZE, oversample=False, shuffle_files=False)
test_ds  = make_dataset(test_files,  SAMPLES_DIR, BATCH_SIZE, oversample=False, shuffle_files=False)

# 4) Fit the model for a small demonstration
print(f"\n>>> Starting demonstration training for {EPOCHS} epochs ...")
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=val_ds,
    validation_steps=VAL_STEPS
)

# 5) Evaluate on test dataset
print("\n>>> Evaluating on test dataset ...")
test_metrics = model.evaluate(
    test_ds,
    steps=TEST_STEPS,
    return_dict=True
)
print("Test metrics:", test_metrics)

print("\nAll done. This single cell handles:")
print(" - Split loading, chunk generator, oversampling, debug prints.")
print(" - Custom focal loss for imbalance, gradient clipping, and ConvLSTM.")
print(" - NaN replacement to avoid nan in forward pass.")


[INFO] Loaded split info from: splits/split_info.json
  => Train files: 108
  => Val   files: 12
  => Test  files: 35
[INFO] Detected channel dimension = 24


I0000 00:00:1747201837.201665    2481 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 94412 MB memory:  -> device: 0, name: NVIDIA GH200 480GB, pci bus id: 0000:dd:00.0, compute capability: 9.0


Model: "Stronger_ConvLSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_ConvLSTM (InputLayer)     │ (None, 24, 3, 128,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Permute (Permute)               │ (None, 3, 128, 128,    │             0 │
│                                 │ 24)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ConvLSTM_1 (ConvLSTM2D)         │ (None, 3, 128, 128,    │        64,640 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BN_1 (BatchNormalization)       │ (None, 3, 128, 128,    │           128 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ConvLSTM_2 (ConvLSTM2D)         │ (None, 128, 128, 32)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BN_2 (BatchNormalization)       │ (None, 128, 128, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Flatten (Flatten)               │ (None, 524288)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense_128 (Dense)               │ (None, 128)            │    67,108,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 67,247,873 (256.53 MB)

...


>>> Starting demonstration training for 2 epochs ...
Epoch 1/2

[DEBUG] Loading samples_2017_09.npz, x_data.shape=(2051, 24, 3, 128, 128)
   Debug patch idx=1124, channel=10, time=0, NaN%=0.00, label=0
   Debug patch idx=188, channel=14, time=0, NaN%=0.00, label=0
   Debug patch idx=1727, channel=11, time=0, NaN%=0.00, label=0


I0000 00:00:1747201887.438677    4969 service.cc:148] XLA service 0xe531dc00de40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747201887.438709    4969 service.cc:156]   StreamExecutor device (0): NVIDIA GH200 480GB, Compute Capability 9.0
2025-05-14 05:51:27.541304: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747201888.026067    4969 cuda_dnn.cc:529] Loaded cuDNN version 90701


 1/50 ━━━━━━━━━━━━━━━━━━━━ 35:17 43s/step - accuracy: 0.5156 - loss: 0.2261

I0000 00:00:1747201891.123502    4969 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


48/50 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 0.6716 - loss: 1.7540
[DEBUG] Loading samples_2014_08.npz, x_data.shape=(2051, 24, 3, 128, 128)
   Debug patch idx=1300, channel=6, time=2, NaN%=0.00, label=1
   Debug patch idx=1373, channel=14, time=2, NaN%=0.00, label=0
   Debug patch idx=451, channel=6, time=1, NaN%=0.00, label=0
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 853ms/step - accuracy: 0.6728 - loss: 1.7536
[DEBUG] Loading samples_2021_01.npz, x_data.shape=(2051, 24, 3, 128, 128)
   Debug patch idx=606, channel=18, time=0, NaN%=0.00, label=0
   Debug patch idx=2033, channel=10, time=0, NaN%=0.00, label=0
   Debug patch idx=208, channel=23, time=0, NaN%=0.00, label=0
50/50 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - accuracy: 0.6734 - loss: 1.7535 - val_accuracy: 0.8625 - val_loss: 0.9729
Epoch 2/2
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 107ms/step - accuracy: 0.7284 - loss: 1.8361 - val_accuracy: 0.8641 - val_loss: 0.9988

[DEBUG] Loading samples_2018_11.npz, x_data.shape=(2051, 24, 3, 128, 128)
   De

In [4]:
# ===========================================================================================
# SINGLE-CELL NOTEBOOK SCRIPT
# Includes:
#   - Split loading
#   - Generator with oversampling & debug
#   - Strong ConvLSTM + Focal Loss
#   - Validation confusion matrix after each epoch
#   - Final test confusion matrix
# ===========================================================================================

import os
import json
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# ----------------------------------------------------------------------------
# 1) Configuration
# ----------------------------------------------------------------------------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

SPLITS_DIR   = "splits"           # Folder containing split_info.json
SPLIT_FILE   = "split_info.json"  # The split file name
SAMPLES_DIR  = "Stage1_Samples"   # Folder with chunk .npz

MAX_RAM_USAGE = 15_000_000_000    # ~15 GB memory threshold
BATCH_SIZE    = 64

EPOCHS          = 5   
STEPS_PER_EPOCH = 50
VAL_STEPS       = 10
TEST_STEPS      = 10

# How many steps to compute confusion matrix on validation set after each epoch
VAL_CM_STEPS    = 10   # partial coverage
TEST_CM_STEPS   = 10

# ----------------------------------------------------------------------------
# 2) Load Splits
# ----------------------------------------------------------------------------
def load_splits(splits_dir=SPLITS_DIR, split_file=SPLIT_FILE):
    path = os.path.join(splits_dir, split_file)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Split file not found: {path}")
    with open(path, "r") as f:
        info = json.load(f)
    return info

split_info = load_splits()
train_files = split_info["splits"]["train"]
val_files   = split_info["splits"]["validation"]
test_files  = split_info["splits"]["test"]

print("[INFO] Loaded split info from:", os.path.join(SPLITS_DIR, SPLIT_FILE))
print(f"  => Train files: {len(train_files)}")
print(f"  => Val   files: {len(val_files)}")
print(f"  => Test  files: {len(test_files)}")

# ----------------------------------------------------------------------------
# 3) Generator with Oversampling, Debug Prints, Memory Check
# ----------------------------------------------------------------------------
def debug_print_sample_slices(x_data, y_data, num_checks=3):
    """Print random patch slices for NaN ratio & label."""
    if x_data.shape[0] == 0:
        return
    idx = np.random.choice(x_data.shape[0], size=min(num_checks, x_data.shape[0]), replace=False)
    for i in idx:
        c = np.random.randint(0, x_data.shape[1])  # random channel
        t = np.random.randint(0, x_data.shape[2])  # random time step
        patch_slice = x_data[i, c, t]  # [128,128]
        nan_count = np.isnan(patch_slice).sum()
        nan_pct = (nan_count / (128*128)) * 100.0
        label_val = y_data[i]
        print(f"   Debug idx={i}, ch={c}, t={t}, NaN%={nan_pct:.2f}, label={label_val}")

def chunk_file_generator(file_list, samples_dir, batch_size,
                         oversample=True, shuffle_files=True, debug_checks=3):
    """Loads chunk files in random order, prints debug, oversamples, yields mini-batches."""
    files = list(file_list)
    if shuffle_files:
        random.shuffle(files)

    for fname in files:
        fpath = os.path.join(samples_dir, fname)
        if not os.path.exists(fpath):
            raise FileNotFoundError(f"Chunk file not found: {fpath}")

        with np.load(fpath, allow_pickle=True) as data:
            x_data = data["x_data"]  # [N, channels, time=3, 128,128]
            y_data = data["y_label"] # [N]

        # Memory check
        approx_mem = x_data.size * x_data.itemsize + y_data.size * y_data.itemsize
        if approx_mem > MAX_RAM_USAGE:
            print(f"[Warning] chunk {fname} => ~{approx_mem/1e9:.2f} GB > limit {MAX_RAM_USAGE/1e9:.2f} GB")

        print(f"\n[DEBUG] Loading {fname}, shape={x_data.shape}")
        debug_print_sample_slices(x_data, y_data, debug_checks)

        # Replace NaNs
        x_data = np.nan_to_num(x_data, nan=0.0)

        # Oversample minority class
        if oversample:
            pos_idx = np.where(y_data == 1)[0]
            neg_idx = np.where(y_data == 0)[0]
            if len(pos_idx) > 0 and len(neg_idx) > 0:
                if len(pos_idx) < len(neg_idx):
                    factor = len(neg_idx) // len(pos_idx)
                    remainder = len(neg_idx) % len(pos_idx)
                    replicated = np.concatenate(
                        [pos_idx]*factor +
                        ([np.random.choice(pos_idx, remainder, replace=False)] if remainder>0 else [])
                    )
                    new_idx = np.concatenate([neg_idx, replicated])
                else:
                    factor = len(pos_idx) // len(neg_idx)
                    remainder = len(pos_idx) % len(neg_idx)
                    replicated = np.concatenate(
                        [neg_idx]*factor +
                        ([np.random.choice(neg_idx, remainder, replace=False)] if remainder>0 else [])
                    )
                    new_idx = np.concatenate([pos_idx, replicated])
                np.random.shuffle(new_idx)
                x_data = x_data[new_idx]
                y_data = y_data[new_idx]

        # Shuffle for mini-batching
        indices = np.arange(x_data.shape[0])
        np.random.shuffle(indices)

        start = 0
        while start < len(indices):
            end = min(start + batch_size, len(indices))
            batch_idx = indices[start:end]
            start = end

            yield x_data[batch_idx], y_data[batch_idx]

        del x_data, y_data

def make_dataset(file_list, samples_dir, batch_size, oversample=True, shuffle_files=True):
    def gen():
        return chunk_file_generator(
            file_list=file_list,
            samples_dir=samples_dir,
            batch_size=batch_size,
            oversample=oversample,
            shuffle_files=shuffle_files,
            debug_checks=3
        )
    output_types = (tf.float32, tf.int32)
    output_shapes = (
        tf.TensorShape([None, None, 3, 128, 128]),
        tf.TensorShape([None])
    )
    return tf.data.Dataset.from_generator(gen, output_types, output_shapes)

# ----------------------------------------------------------------------------
# 4) ConvLSTM + Focal Loss
# ----------------------------------------------------------------------------
def build_stronger_convlstm(channels):
    """
    A deeper ConvLSTM model with two ConvLSTM2D layers, BN, Flatten, Dense, Dropout
    """
    inp = layers.Input(shape=(channels, 3, 128, 128), name="Input_ConvLSTM")
    x = layers.Permute((2,3,4,1), name="Permute")(inp)  # => (batch, time=3, h=128, w=128, channels)

    x = layers.ConvLSTM2D(32, (3,3), padding="same", activation="tanh", return_sequences=True)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ConvLSTM2D(32, (3,3), padding="same", activation="tanh", return_sequences=False)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(1, activation="sigmoid")(x)

    return models.Model(inp, out, name="Stronger_ConvLSTM")

def custom_sigmoid_focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):
    """Focal loss for binary classification, helps handle class imbalance."""
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1.0 - 1e-7)
    y_true = tf.cast(y_true, y_pred.dtype)

    ce = tf.keras.backend.binary_crossentropy(y_true, y_pred, from_logits=False)
    p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
    alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
    modulating_factor = tf.pow(1.0 - p_t, gamma)
    return tf.reduce_mean(alpha_factor * modulating_factor * ce)

# ----------------------------------------------------------------------------
# 5) Callback: Print confusion matrix on val set after each epoch
# ----------------------------------------------------------------------------
class ConfusionMatrixCallback(tf.keras.callbacks.Callback):
    """
    After each epoch, runs through 'val_dataset' up to 'val_steps' batches,
    collects predictions, prints confusion matrix.
    """
    def __init__(self, val_dataset, val_steps, batch_size=64):
        super().__init__()
        self.val_dataset = val_dataset
        self.val_steps = val_steps
        self.batch_size = batch_size

    def on_epoch_end(self, epoch, logs=None):
        all_preds = []
        all_labels= []
        steps_count = 0
        for x_batch, y_batch in self.val_dataset:
            preds = self.model.predict(x_batch, verbose=0)
            preds_bin = (preds > 0.5).astype(np.int32)

            all_preds.append(preds_bin)
            all_labels.append(y_batch)

            steps_count += 1
            if (self.val_steps is not None) and (steps_count >= self.val_steps):
                break

        if len(all_preds) > 0:
            all_preds = np.concatenate(all_preds, axis=0).flatten()
            all_labels= np.concatenate(all_labels, axis=0).flatten()
            cm = tf.math.confusion_matrix(labels=all_labels, predictions=all_preds, num_classes=2)
            print(f"\nConfusion Matrix (Val) After Epoch {epoch+1}:\n{cm.numpy()}\n")

# ----------------------------------------------------------------------------
# 6) Build Model, Datasets & Train
# ----------------------------------------------------------------------------

# Detect channel dimension
if len(train_files) == 0:
    raise ValueError("No training files in 'train' split.")
first_train_file = os.path.join(SAMPLES_DIR, train_files[0])
with np.load(first_train_file, allow_pickle=True) as data:
    x_sample = data["x_data"]
detected_channels = x_sample.shape[1]
print(f"[INFO] Detected channel dimension = {detected_channels}")

# Build & compile model
model = build_stronger_convlstm(detected_channels)
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4, clipnorm=5.0),
    loss=lambda yt, yp: custom_sigmoid_focal_loss(yt, yp, gamma=2.0, alpha=0.25),
    metrics=["accuracy"]
)
model.summary()

# Create train/val/test datasets
train_ds = make_dataset(train_files, SAMPLES_DIR, BATCH_SIZE, oversample=True,  shuffle_files=True)
val_ds   = make_dataset(val_files,   SAMPLES_DIR, BATCH_SIZE, oversample=False, shuffle_files=False)
test_ds  = make_dataset(test_files,  SAMPLES_DIR, BATCH_SIZE, oversample=False, shuffle_files=False)

# Confusion matrix callback for validation
cm_callback = ConfusionMatrixCallback(val_dataset=val_ds, val_steps=VAL_CM_STEPS)

# Train with confusion-matrix callback
print(f"\n>>> Starting demonstration training for {EPOCHS} epochs ...")
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=val_ds,
    validation_steps=VAL_STEPS,
    callbacks=[cm_callback]
)

# Evaluate on test set
print("\n>>> Evaluating on test dataset ...")
test_result = model.evaluate(test_ds, steps=TEST_STEPS, return_dict=True)
print("Test metrics:", test_result)

# Print confusion matrix on test set
all_test_preds = []
all_test_labels= []
steps_count = 0
for x_batch, y_batch in test_ds:
    pred = model.predict(x_batch, verbose=0)
    pred_bin = (pred > 0.5).astype(np.int32)

    all_test_preds.append(pred_bin)
    all_test_labels.append(y_batch)

    steps_count += 1
    if (TEST_CM_STEPS is not None) and (steps_count >= TEST_CM_STEPS):
        break

if len(all_test_preds) > 0:
    all_test_preds = np.concatenate(all_test_preds).flatten()
    all_test_labels= np.concatenate(all_test_labels).flatten()
    test_cm = tf.math.confusion_matrix(labels=all_test_labels, predictions=all_test_preds, num_classes=2)
    print("\nFinal Test Confusion Matrix:\n", test_cm.numpy())

print("\nAll done. We now print a confusion matrix after every epoch on the validation set, and finally on the test set.")


[INFO] Loaded split info from: splits/split_info.json
  => Train files: 108
  => Val   files: 12
  => Test  files: 35
[INFO] Detected channel dimension = 24


Model: "Stronger_ConvLSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Input_ConvLSTM (InputLayer)     │ (None, 24, 3, 128,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Permute (Permute)               │ (None, 3, 128, 128,    │             0 │
│                                 │ 24)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_2 (ConvLSTM2D)      │ (None, 3, 128, 128,    │        64,640 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 3, 128, 128,    │           128 │
│ (BatchNormalization)            │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_3 (ConvLSTM2D)      │ (None, 128, 128, 32)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 524288)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    67,108,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 67,247,873 (256.53 MB)

 Trainable params: 67,247,745 (256.53 MB)

 Non-trainable params: 128 (512.00 B)


>>> Starting demonstration training for 5 epochs ...
Epoch 1/5

[DEBUG] Loading samples_2017_09.npz, shape=(2051, 24, 3, 128, 128)
   Debug idx=1124, ch=10, t=0, NaN%=0.00, label=0
   Debug idx=188, ch=14, t=0, NaN%=0.00, label=0
   Debug idx=1727, ch=11, t=0, NaN%=0.00, label=0
48/50 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.6716 - loss: 1.7540
[DEBUG] Loading samples_2014_08.npz, shape=(2051, 24, 3, 128, 128)
   Debug idx=1300, ch=6, t=2, NaN%=0.00, label=1
   Debug idx=1373, ch=14, t=2, NaN%=0.00, label=0
   Debug idx=451, ch=6, t=1, NaN%=0.00, label=0
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 841ms/step - accuracy: 0.6728 - loss: 1.7536
[DEBUG] Loading samples_2021_01.npz, shape=(2051, 24, 3, 128, 128)
   Debug idx=606, ch=18, t=0, NaN%=0.00, label=0
   Debug idx=2033, ch=10, t=0, NaN%=0.00, label=0
   Debug idx=208, ch=23, t=0, NaN%=0.00, label=0

[DEBUG] Loading samples_2021_01.npz, shape=(2051, 24, 3, 128, 128)
   Debug idx=1515, ch=19, t=1, NaN%=0.00, label=0
   Debug idx=1185, ch=

In [4]:
# ============================================================================================
# ADVANCED SPATIOTEMPORAL MODEL (TimeDistributed CNN then Transformer) + Weighted Focal Loss
# NO PLUS SIGNS in layer/model names to fix scope name errors
# ============================================================================================
import os
import json
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# ----------------------------------------------------------------------------
# [1] Configuration
# ----------------------------------------------------------------------------
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

SPLITS_DIR    = "splits"
SPLIT_FILE    = "split_info.json"
SAMPLES_DIR   = "Stage1_Samples"
MAX_RAM_USAGE = 10_000_000_000  # ~10GB
BATCH_SIZE    = 64

EPOCHS          = 5
STEPS_PER_EPOCH = 50
VAL_STEPS       = 10
TEST_STEPS      = 10

# For confusion matrix partial coverage
VAL_CM_STEPS  = 10
TEST_CM_STEPS = 10

# ----------------------------------------------------------------------------
# [2] Load Splits
# ----------------------------------------------------------------------------
def load_splits(splits_dir=SPLITS_DIR, split_file=SPLIT_FILE):
    path = os.path.join(splits_dir, split_file)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Split file not found: {path}")
    with open(path, "r") as f:
        info = json.load(f)
    return info

split_info = load_splits()
train_files = split_info["splits"]["train"]
val_files   = split_info["splits"]["validation"]
test_files  = split_info["splits"]["test"]

print("[INFO] Loaded split info from:", os.path.join(SPLITS_DIR, SPLIT_FILE))
print(f"  => Train files: {len(train_files)}")
print(f"  => Val   files: {len(val_files)}")
print(f"  => Test  files: {len(test_files)}")

# ----------------------------------------------------------------------------
# [3] Generator
# ----------------------------------------------------------------------------
def debug_print_sample_slices(x_data, y_data, num_checks=3):
    """Shows random patches for NaN% and label."""
    if x_data.shape[0] == 0:
        return
    chosen = np.random.choice(x_data.shape[0], size=min(num_checks, x_data.shape[0]), replace=False)
    for i in chosen:
        c = np.random.randint(0, x_data.shape[1])  
        t = np.random.randint(0, x_data.shape[2])  
        patch_slice = x_data[i, c, t]  
        nan_count = np.isnan(patch_slice).sum()
        nan_pct   = 100.0 * nan_count / (128*128)
        print(f"   Debug idx={i}, ch={c}, t={t}, NaN%={nan_pct:.2f}, label={y_data[i]}")

def chunk_file_generator(file_list, samples_dir, batch_size,
                         oversample=True, shuffle_files=True, debug_checks=3):
    """
    Loads chunk files, optionally oversamples the minority class,
    debug prints random patches, yields (x_batch, y_batch).
    """
    files = list(file_list)
    if shuffle_files:
        random.shuffle(files)

    for fname in files:
        fpath = os.path.join(samples_dir, fname)
        if not os.path.exists(fpath):
            raise FileNotFoundError(f"Chunk file not found: {fpath}")

        with np.load(fpath, allow_pickle=True) as data:
            x_data = data["x_data"]  # shape [N, channels, time=3, 128,128]
            y_data = data["y_label"] # shape [N]

        approx_mem = x_data.size*x_data.itemsize + y_data.size*y_data.itemsize
        if approx_mem > MAX_RAM_USAGE:
            print(f"[WARNING] chunk {fname} ~{approx_mem/1e9:.2f} GB > limit {MAX_RAM_USAGE/1e9:.2f} GB")

        print(f"\n[DEBUG] Loading {fname}, shape={x_data.shape}")
        debug_print_sample_slices(x_data, y_data, debug_checks)

        x_data = np.nan_to_num(x_data, nan=0.0)

        if oversample:
            pos_idx = np.where(y_data==1)[0]
            neg_idx = np.where(y_data==0)[0]
            if len(pos_idx)>0 and len(neg_idx)>0:
                if len(pos_idx)<len(neg_idx):
                    factor = len(neg_idx)//len(pos_idx)
                    remainder = len(neg_idx)%len(pos_idx)
                    replicated = np.concatenate(
                        [pos_idx]*factor +
                        ([np.random.choice(pos_idx, remainder, replace=False)] if remainder>0 else [])
                    )
                    new_idx = np.concatenate([neg_idx, replicated])
                else:
                    factor = len(pos_idx)//len(neg_idx)
                    remainder = len(pos_idx)%len(neg_idx)
                    replicated = np.concatenate(
                        [neg_idx]*factor +
                        ([np.random.choice(neg_idx, remainder, replace=False)] if remainder>0 else [])
                    )
                    new_idx = np.concatenate([pos_idx, replicated])
                np.random.shuffle(new_idx)
                x_data = x_data[new_idx]
                y_data = y_data[new_idx]

        idxs = np.arange(x_data.shape[0])
        np.random.shuffle(idxs)
        start=0
        while start<len(idxs):
            end = min(start+batch_size, len(idxs))
            b_idx= idxs[start:end]
            start= end
            yield x_data[b_idx], y_data[b_idx]

        del x_data, y_data

def make_dataset(file_list, samples_dir, batch_size, oversample=True, shuffle_files=True):
    def gen():
        return chunk_file_generator(
            file_list=file_list,
            samples_dir=samples_dir,
            batch_size=batch_size,
            oversample=oversample,
            shuffle_files=shuffle_files,
            debug_checks=3
        )
    out_types = (tf.float32, tf.int32)
    out_shapes= (
        tf.TensorShape([None, None, 3, 128, 128]),
        tf.TensorShape([None])
    )
    return tf.data.Dataset.from_generator(gen, out_types, out_shapes)

# ----------------------------------------------------------------------------
# [4] Weighted Focal Loss
# ----------------------------------------------------------------------------
def weighted_focal_loss(y_true, y_pred, gamma=2.0, alpha=0.75):
    """
    Weighted Focal Loss: alpha=0.75 => heavily bias to minority class.
    """
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1-1e-7)
    y_true = tf.cast(y_true, y_pred.dtype)
    ce = tf.keras.backend.binary_crossentropy(y_true, y_pred, from_logits=False)
    p_t = y_true*y_pred + (1-y_true)*(1-y_pred)
    alpha_factor = y_true*alpha + (1-y_true)*(1-alpha)
    mod_factor   = tf.pow(1.0 - p_t, gamma)
    return tf.reduce_mean(alpha_factor * mod_factor * ce)

# ----------------------------------------------------------------------------
# [5] TimeDistributed CNN + Transformer
# ----------------------------------------------------------------------------
def build_transformer_model(channels, seq_len=3):
    """
    1) Input => (channels, time=3, 128,128)
    2) Permute => (batch, time=3, 128,128, channels)
    3) TimeDistCnnBlock => embedding for each time
    4) Transformer block => multi-head self-attention across time
    5) Global average => Dense => Sigmoid
    """
    inp = layers.Input(shape=(channels, seq_len, 128,128), name="InputLayer")

    x = layers.Permute((2,3,4,1), name="PermuteLayer")(inp)  
    # => (batch, time=3, 128,128, channels)

    # A small CNN block for each time step
    def make_cnn_block():
        block = models.Sequential(name="TimeCnnBlock")
        block.add(layers.Conv2D(16, (3,3), padding="same", activation="relu"))
        block.add(layers.MaxPooling2D((2,2)))  # => 64x64
        block.add(layers.Conv2D(32, (3,3), padding="same", activation="relu"))
        block.add(layers.MaxPooling2D((2,2)))  # => 32x32
        block.add(layers.Conv2D(64, (3,3), padding="same", activation="relu"))
        block.add(layers.MaxPooling2D((2,2)))  # => 16x16
        block.add(layers.Flatten())
        block.add(layers.Dense(256, activation="relu"))
        return block

    time_cnn = make_cnn_block()
    x = layers.TimeDistributed(time_cnn, name="TimeDistCnnLayer")(x)
    # => shape [batch, time=3, 256]

    # Transformer
    embed_dim = 256
    num_heads = 4
    ff_dim    = 256

    attn_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name="MHA")(x, x)
    x = layers.Add(name="AttnAdd")([x, attn_out])
    x = layers.LayerNormalization(name="AttnLN")(x)

    ff = layers.Dense(ff_dim, activation="relu", name="FF1")(x)
    ff = layers.Dense(embed_dim, name="FF2")(ff)
    x  = layers.Add(name="FFAdd")([x, ff])
    x  = layers.LayerNormalization(name="FFLN")(x)

    # Global average over time => [batch, embed_dim]
    x = layers.GlobalAveragePooling1D(name="TimeGAP")(x)

    x = layers.Dense(128, activation="relu", name="Dense128")(x)
    x = layers.Dropout(0.3, name="Dropout")(x)
    out= layers.Dense(1, activation="sigmoid", name="Output")(x)

    return models.Model(inp, out, name="CnnTransformerModel")

# ----------------------------------------------------------------------------
# [6] ConfusionMatrixCallback
# ----------------------------------------------------------------------------
class ConfusionMatrixCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_dataset, val_steps):
        super().__init__()
        self.val_dataset= val_dataset
        self.val_steps  = val_steps

    def on_epoch_end(self, epoch, logs=None):
        all_preds, all_labels = [], []
        step_count=0
        for x_b, y_b in self.val_dataset:
            preds = self.model.predict(x_b, verbose=0)
            preds_bin = (preds>0.5).astype(np.int32)
            all_preds.append(preds_bin)
            all_labels.append(y_b)
            step_count+=1
            if self.val_steps is not None and step_count>=self.val_steps:
                break
        if len(all_preds)>0:
            all_preds  = np.concatenate(all_preds).flatten()
            all_labels = np.concatenate(all_labels).flatten()
            cm = tf.math.confusion_matrix(all_labels, all_preds, num_classes=2)
            print(f"\nConfusion Matrix (Val) After Epoch {epoch+1}:\n{cm.numpy()}\n")

# ----------------------------------------------------------------------------
# [7] Build, Train, Evaluate
# ----------------------------------------------------------------------------
if len(train_files)==0:
    raise ValueError("No train files found in 'train' split_info")

# 1) Detect channels
first_train_file = os.path.join(SAMPLES_DIR, train_files[0])
with np.load(first_train_file, allow_pickle=True) as data:
    x_sample = data["x_data"]
detected_channels= x_sample.shape[1]
print(f"[INFO] Detected channels={detected_channels}")

# 2) Build the model with safe naming
model = build_transformer_model(detected_channels, seq_len=3)
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4, clipnorm=5.0),
    loss=lambda y_t, y_p: weighted_focal_loss(y_t, y_p, gamma=2.0, alpha=0.75),
    metrics=["accuracy"]
)
model.summary()

# 3) Create Datasets
train_ds = make_dataset(train_files, SAMPLES_DIR, BATCH_SIZE, oversample=True,  shuffle_files=True)
val_ds   = make_dataset(val_files,   SAMPLES_DIR, BATCH_SIZE, oversample=False, shuffle_files=False)
test_ds  = make_dataset(test_files,  SAMPLES_DIR, BATCH_SIZE, oversample=False, shuffle_files=False)

# 4) Callback for confusion matrix
cm_callback = ConfusionMatrixCallback(val_dataset=val_ds, val_steps=VAL_CM_STEPS)

# 5) Train
print(f"\n>>> Starting training for {EPOCHS} epochs [CnnTransformerModel + Weighted Focal Loss] ...")
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=val_ds,
    validation_steps=VAL_STEPS,
    callbacks=[cm_callback]
)

# 6) Evaluate on test set
print("\n>>> Evaluating on test dataset ...")
test_result = model.evaluate(test_ds, steps=TEST_STEPS, return_dict=True)
print("Test metrics:", test_result)

# 7) Final confusion matrix on test
all_test_preds, all_test_labels = [], []
steps_count=0
for x_b, y_b in test_ds:
    pred = model.predict(x_b, verbose=0)
    pred_bin = (pred>0.5).astype(np.int32)
    all_test_preds.append(pred_bin)
    all_test_labels.append(y_b)
    steps_count+=1
    if (TEST_CM_STEPS is not None) and (steps_count>=TEST_CM_STEPS):
        break

if len(all_test_preds)>0:
    all_test_preds  = np.concatenate(all_test_preds).flatten()
    all_test_labels = np.concatenate(all_test_labels).flatten()
    test_cm = tf.math.confusion_matrix(all_test_labels, all_test_preds, num_classes=2)
    print("\nFinal Test Confusion Matrix:\n", test_cm.numpy())

print("\nDone! All layer/model names are free of plus signs, so no scope name errors should arise.")


[INFO] Loaded split info from: splits/split_info.json
  => Train files: 108
  => Val   files: 12
  => Test  files: 35
[INFO] Detected channels=24


Model: "CnnTransformerModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ InputLayer          │ (None, 24, 3,     │          0 │ -                 │
│ (InputLayer)        │ 128, 128)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ PermuteLayer        │ (None, 3, 128,    │          0 │ InputLayer[0][0]  │
│ (Permute)           │ 128, 24)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TimeDistCnnLayer    │ (None, 3, 256)    │  4,221,168 │ PermuteLayer[0][… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ MHA                 │ (None, 3, 256)    │  1,051,904 │ TimeDistCnnLayer… │
│ (MultiHeadAttentio… │                   │            │ TimeDistCnnLayer… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ AttnAdd (Add)       │ (None, 3, 256)    │          0 │ TimeDistCnnLayer… │
│                     │                   │            │ MHA[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ AttnLN              │ (None, 3, 256)    │        512 │ AttnAdd[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ FF1 (Dense)         │ (None, 3, 256)    │     65,792 │ AttnLN[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ FF2 (Dense)         │ (None, 3, 256)    │     65,792 │ FF1[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ FFAdd (Add)         │ (None, 3, 256)    │          0 │ AttnLN[0][0],     │
│                     │                   │            │ FF2[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ FFLN                │ (None, 3, 256)    │        512 │ FFAdd[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TimeGAP             │ (None, 256)       │          0 │ FFLN[0][0]        │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense128 (Dense)    │ (None, 128)       │     32,896 │ TimeGAP[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dropout (Dropout)   │ (None, 128)       │          0 │ Dense128[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Output (Dense)      │ (None, 1)         │        129 │ Dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,438,705 (20.75 MB)

 Trainable params: 5,438,705 (20.75 MB)

 Non-trainable params: 0 (0.00 B)


>>> Starting training for 5 epochs [CnnTransformerModel + Weighted Focal Loss] ...
Epoch 1/5


ValueError: 'CNN+Transformer_TimeDistCnnLayer_TimeCnnBlock_conv2d_9_kernel_momentum' is not a valid scope name. A scope name has to match the following pattern: ^[A-Za-z0-9_.\\/>-]*$